In [3]:
from neo4j import GraphDatabase

In [4]:
class BlockChainDB(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)#REVISE LATER FOR ENCRYPTION!!!

    def close(self):
        self._driver.close()
        
        
    @staticmethod
    def _new_address(tx, address,i,change_addr):
        if change_addr: addr_type = "change"
        else: addr_type = "recipient"
        result = tx.run("CREATE a = (:address {address:$address, acc_index:$index, type:$kind, created:timestamp()}) "
                        "RETURN a ", address=address, index = i, kind = addr_type)
        return result.single()
    
    
    @staticmethod
    def _new_output(tx, amount, script_pubkey,tx_id,out_index):
        result = tx.run("MATCH (n:transaction {id : $tx_id}) "
                        "CREATE p = (:output {out_index:$out_index, script_pubkey=$script_pubkey})<-[:CREATES]-(n) "
                        "RETURN p ", tx_id = tx_id, script_pubkey=script_pubkey, out_index=out_index, local_index=local_index)
        return result.single()
    
    
    @staticmethod
    def _new_tx(version, locktime, tx_id, inputs, outputs, segwit):
        result = tx.run("CREATE (n:transaction {id:$tx_id, version:$version, segwit:$segwit, locktime=$locktime}) "
                        "RETURN n",
                        tx_id=tx_id, segwit=segwit, version=version, locktime=locktime)
        
        for index,tx_out in enumerate(outputs):
            self._new_output(tx_out.amount, tx_out.script_pubkey, tx_id, index)
            
        for tx_in in inputs:
            result = tx.run("MATCH (t:transaction {id:$tx_id}), (o:output {index:$input_index})--(:transaction {id:$prev_tx}) "
                        "CREATE (t)-[:SPENT]->(o)",
                        tx_id=tx_id  input_index=tx_in.prev_index, prev_tx=tx.prev_tx)
        
        return result.single()
    
    
    @staticmethod
    def _new_block(block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        _height = tx.run( "MATCH (u:block) RETURN COUNT (u) ").single()[0] 
        result = tx.run("CREATE (n:block {id:$block_id, version:$version , prev_block:$prev_block, merkle_root:$merkle_root, timestamp:$timestamp, bits:$bits, nonce:$nonce, n_tx:n_tx, height=_height})<-[r:LINKS]-(p:block{id:$prev_block}) "
                        "RETURN n,r,p",
                        block_id=block_id, version=version, prev_block=prev_block, merkle_root=merkle_root, timestamp=timestamp, 
                        bits=bits, nonce=nonce, n_tx=n_tx)
        return result.single()

    def new_address(self, address,i,change_addr):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_address, address,i,change_addr)
            print(result)
            
    def new_output(self, address,tx_id,out_index):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_utxo, address,tx_id,out_index)
            print(result)
            
    def new_tx(self, version, locktime, tx_id, inputs, outputs, segwit):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_tx, tx_id,utxo_local_index_list, outputs)
            print(result)
            
    def new_block(self,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_block,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx)
            print(result)
  
      

In [5]:
db = WalletDB( "neo4j://localhost:7687" , "neo4j" , "wallet" )


OSError: [Errno 0] Error

In [140]:
db.new_address("test_address1",0,False)
db.new_address("test_address2",777,False)
db.new_address("test_address3",22222,False)
db.new_address("test_address4",92747,True)
db.new_address("test_address5",99199191,True)
db.new_address("test_address6",666,True)
db.new_address("test_address7",625,True)

<Record a=<Path start=<Node id=14 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'acc_index': 0, 'created': 1586244466132}> end=<Node id=14 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'acc_index': 0, 'created': 1586244466132}> size=0>>
<Record a=<Path start=<Node id=15 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'acc_index': 777, 'created': 1586244466145}> end=<Node id=15 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'acc_index': 777, 'created': 1586244466145}> size=0>>
<Record a=<Path start=<Node id=0 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'acc_index': 22222, 'created': 1586244466152}> end=<Node id=0 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'acc_index': 22222, 'created': 1586244466152}> size=0>>
<Record a=<Path start=<Node id=8 labels={'address'} properties={'address': 

In [141]:
db.new_utxo("test_address1",1000,2)
db.new_utxo("test_address2",1001,0)
db.new_utxo("test_address3",1002,1)
db.new_utxo("test_address4",1003,8)
db.new_utxo("test_address5",1004,5)

<Record p=<Path start=<Node id=6 labels={'utxo'} properties={'transaction_id': 1000, 'out_index': 2, 'local_index': 0, 'address': 'test_address1', 'spent': False}> end=<Node id=14 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'created': 1586244466132, 'acc_index': 0}> size=1>>
<Record p=<Path start=<Node id=7 labels={'utxo'} properties={'transaction_id': 1001, 'out_index': 0, 'local_index': 1, 'address': 'test_address2', 'spent': False}> end=<Node id=15 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'created': 1586244466145, 'acc_index': 777}> size=1>>
<Record p=<Path start=<Node id=9 labels={'utxo'} properties={'transaction_id': 1002, 'out_index': 1, 'local_index': 2, 'address': 'test_address3', 'spent': False}> end=<Node id=0 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'created': 1586244466152, 'acc_index': 22222}> size=1>>
<Record p=<Path start=<Node id=10 labels={'utxo'} propertie

In [142]:
db.new_tx(2222,[0,1], ["ouptut1","output2"])

None


In [143]:
db.update_confirmations(2222,7)

<Record n=<Node id=12 labels={'TxOut'} properties={'outputs': ['ouptut1', 'output2'], 'id': 2222, 'confirmations': 7, 'created': 1586244466250, 'inputs': [0, 1]}>>


In [144]:
up = db.update_utxo(2222)

In [145]:
db.clean_addresses()

None


In [161]:
add = db.look_for_coins()

In [162]:
add

[{'coin.transaction_id': 1002,
  'coin.out_index': 1,
  'coin.address': 'test_address3'},
 {'coin.transaction_id': 1003,
  'coin.out_index': 8,
  'coin.address': 'test_address4'},
 {'coin.transaction_id': 1004,
  'coin.out_index': 5,
  'coin.address': 'test_address5'}]

In [168]:
unused_add = db.get_unused_addresses()
unused_add

[{'unused_address.address': 'test_address6',
  'unused_address.acc_index': 666,
  'unused_address.type': 'change'},
 {'unused_address.address': 'test_address7',
  'unused_address.acc_index': 625,
  'unused_address.type': 'change'}]

In [4]:
driver = GraphDatabase.driver("neo4j://localhost:7687" , auth=("neo4j", "wallet"), encrypted=False)

In [30]:
db.run_as_system()

CypherSyntaxError: Invalid input ':': expected <init> (line 1, column 1 (offset: 0))
":use system RETURN a"
 ^